In [113]:
#load in packages
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [114]:
#Read in dpdcalls data set
dpdcalls = gpd.read_file('/Users/sadiesheridan/Desktop/dpdshapefile/dpdshapefile.shp')
dpdcalls.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [115]:
#Read in policebeats data set
policebeats = gpd.read_file('/Users/sadiesheridan/Desktop/Police_Beats/Police_Beats.shp')
policebeats.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [116]:
#Print .info() for policebeats
policebeats.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    119 non-null    int64   
 1   PERIMETER   119 non-null    float64 
 2   STATUS      83 non-null     object  
 3   CONTIGUOUS  83 non-null     object  
 4   LAWDIST     119 non-null    object  
 5   LAWBEAT     119 non-null    int64   
 6   CAD         119 non-null    object  
 7   GIS_DataA1  0 non-null      object  
 8   ShapeSTAre  119 non-null    float64 
 9   ShapeSTLen  119 non-null    float64 
 10  geometry    119 non-null    geometry
dtypes: float64(3), geometry(1), int64(2), object(5)
memory usage: 10.4+ KB


In [117]:
#Print .info() for dpdcalls
dpdcalls.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 331077 entries, 0 to 331076
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Event Numb  331077 non-null  int64   
 1   Case Numbe  45954 non-null   object  
 2   Call Date   331077 non-null  object  
 3   Call Time   331077 non-null  object  
 4   Source      331077 non-null  object  
 5   Priority    330425 non-null  object  
 6   Nature      331076 non-null  object  
 7   Address     330437 non-null  object  
 8   X           331077 non-null  int64   
 9   Y           331077 non-null  int64   
 10  District    319334 non-null  object  
 11  Beat        319315 non-null  object  
 12  Dispositio  331077 non-null  object  
 13  Cancelled   331077 non-null  int64   
 14  geometry    331077 non-null  geometry
dtypes: geometry(1), int64(4), object(10)
memory usage: 37.9+ MB


In [118]:
#Print null values with df.isna().sum() function
dpdcalls.isna().sum()

Event Numb         0
Case Numbe    285123
Call Date          0
Call Time          0
Source             0
Priority         652
Nature             1
Address          640
X                  0
Y                  0
District       11743
Beat           11762
Dispositio         0
Cancelled          0
geometry           0
dtype: int64

In [119]:
#Looking at records with coordinates zero
dpdcalls.loc[(dpdcalls["X"] == 0) | (dpdcalls["Y"] == 0)]

,Event Numb,Case Numbe,Call Date,Call Time,Source,Priority,Nature,Address,X,Y,District,Beat,Dispositio,Cancelled,geometry
127,21000194,None,1/1/2021,1:25:25AM,Self Initiated,4,ATTEMPT TO LOCATE,CARELESS AND RECKLESS AT 11 147 NB,0,0,None,None,(blank),0,POINT (-85.56829 33.56815)
131,21000199,None,1/1/2021,1:29:04AM,Self Initiated,4,ATTEMPT TO LOCATE,CARELESS AND RECKLESS AT 147S AND BLACKWELL ST,0,0,None,None,(blank),0,POINT (-85.56829 33.56815)
210,21000322,None,1/1/2021,3:56:22AM,Self Initiated,5,DIRECTED PATROL,3407 3RD FORK RD,0,0,None,None,Cancelled,1,POINT (-85.56829 33.56815)
255,21000381,None,1/1/2021,6:14:49AM,Self Initiated,4,ATTEMPT TO LOCATE,CARELESS AND RECKLESS AT 1010 N DURHAM FREEWAY,0,0,None,None,(blank),0,POINT (-85.56829 33.56815)
274,21000409,None,1/1/2021,7:03:23AM,Self Initiated,4,FOLLOW UP,SS1,0,0,None,None,Resolved Without Report,0,POINT (-85.56829 33.56815)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330699,21477803,21045233,12/31/2021,6:00:48PM,Mobil,4,ATTEMPT TO LOCATE,UNAUTHORIZED USE AT 215 WILLIAM PENN PLAZA APT...,0,0,None,None,(blank),0,POINT (-85.56829 33.56815)
330858,21478039,None,12/31/2021,8:24:41PM,Self Initiated,4,ATTEMPT TO LOCATE,CARELESS AND RECKLESS AT 4623 DURHAM CHAPEL HI...,0,0,None,None,(blank),0,POINT (-85.56829 33.56815)
330963,21478179,None,12/31/2021,10:06:37PM,Self Initiated,4,PROPERTY CHECK,1223 HOMECOMING WY,0,0,None,None,Resolved Without Report,0,POINT (-85.56829 33.56815)
331057,21478324,None,12/31/2021,11:45:43PM,Self Initiated,4,ATTEMPT TO LOCATE,CARELESS AND RECKLESS AT 17900 I85 S,0,0,None,None,(blank),0,POINT (-85.56829 33.56815)


In [120]:
#Filter out null x and y values
dpdcalls = dpdcalls.loc[~(dpdcalls["X"] == 0) & ~(dpdcalls["Y"] == 0)].copy()
dpdcalls.shape

(322145, 15)

In [121]:
#Filter out cancelled calls and self initiated calls
dpdcalls = dpdcalls.loc[dpdcalls["Source"] != "Self Initiated"].copy()
dpdcalls = dpdcalls.loc[dpdcalls["Cancelled"] == 0].copy()
dpdcalls.shape

(122803, 15)

In [123]:
#Filter out beats not in Durham for policebeats
policebeats_list = [121, 122, 112, 111, 123, 114, 113, 124, 212, 214, 213, 211, 223, 222, 221, 224, 313, 312, 321, 314, 323, 324, 322, 311, 411, 415, 412, 413, 414, 422, 421, 423, 511, 512, 513]
policebeats = policebeats.loc[policebeats["LAWBEAT"].isin(policebeats_list)].copy()
policebeats.shape

(83, 11)

In [124]:
#Print changes
dpdcalls.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 122803 entries, 0 to 331065
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Event Numb  122803 non-null  int64   
 1   Case Numbe  34318 non-null   object  
 2   Call Date   122803 non-null  object  
 3   Call Time   122803 non-null  object  
 4   Source      122803 non-null  object  
 5   Priority    122258 non-null  object  
 6   Nature      122803 non-null  object  
 7   Address     122803 non-null  object  
 8   X           122803 non-null  int64   
 9   Y           122803 non-null  int64   
 10  District    122148 non-null  object  
 11  Beat        122128 non-null  object  
 12  Dispositio  122803 non-null  object  
 13  Cancelled   122803 non-null  int64   
 14  geometry    122803 non-null  geometry
dtypes: geometry(1), int64(4), object(10)
memory usage: 15.0+ MB


In [129]:
policebeats.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 83 entries, 11 to 118
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    83 non-null     int64   
 1   PERIMETER   83 non-null     float64 
 2   STATUS      83 non-null     object  
 3   CONTIGUOUS  83 non-null     object  
 4   LAWDIST     83 non-null     object  
 5   LAWBEAT     83 non-null     int64   
 6   CAD         83 non-null     object  
 7   GIS_DataA1  0 non-null      object  
 8   ShapeSTAre  83 non-null     float64 
 9   ShapeSTLen  83 non-null     float64 
 10  geometry    83 non-null     geometry
dtypes: float64(3), geometry(1), int64(2), object(5)
memory usage: 7.8+ KB


In [125]:
#Merge Calls and Beats data sets
dpd_w_policebeats = gpd.sjoin(dpdcalls, policebeats)
dpd_w_policebeats

,Event Numb,Case Numbe,Call Date,Call Time,Source,Priority,Nature,Address,X,Y,...,OBJECTID,PERIMETER,STATUS,CONTIGUOUS,LAWDIST,LAWBEAT,CAD,GIS_DataA1,ShapeSTAre,ShapeSTLen
0,21000002,None,1/1/2021,12:00:28AM,Wireless 911 Call,3,SOUND OF SHOTS,100 PEBBLESTONE DR,2055372,818537,...,47,61714.988281,I,yes,D1,121,121,None,3.419525e+07,61714.987980
81,21000125,None,1/1/2021,12:46:53AM,Phone Call,3,SOUND OF SHOTS,PEBBLESTONE DR/KNIGHTWOOD DR,2056514,821822,...,47,61714.988281,I,yes,D1,121,121,None,3.419525e+07,61714.987980
1117,21001615,21000126,1/2/2021,11:30:25AM,Wireless 911 Call,4,DAMAGE TO PROPERTY,0 AUTUMN LEAF LN,2053898,824503,...,47,61714.988281,I,yes,D1,121,121,None,3.419525e+07,61714.987980
1226,21001780,21000150,1/2/2021,2:12:39PM,Wireless 911 Call,4,LARCENY,3600 CARDINAL LAKE DR,2056795,825199,...,47,61714.988281,I,yes,D1,121,121,None,3.419525e+07,61714.987980
1319,21001917,None,1/2/2021,4:50:21PM,Phone Call,3,SUSPICIOUS PERSON,ESTERBROOK CT/PEBBLESTONE DR,2055583,819061,...,47,61714.988281,I,yes,D1,121,121,None,3.419525e+07,61714.987980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286528,21411398,None,11/10/2021,12:57:06PM,Wireless 911 Call,3,ASSAULT,3000 MT SINAI RD,2002584,814666,...,69,4025.813965,I,yes,D3,321,321,None,9.559212e+05,4025.813820
190058,21266548,None,7/25/2021,2:51:58PM,Wireless 911 Call,2,URGENT WELFARE CHECK,5500 LEESVILLE RD,2060394,797620,...,98,1949.288940,I,no,D1,124,124,None,1.472133e+05,1949.288667
264761,21378037,None,10/16/2021,2:45:39AM,Phone Call,2,ASSIST EMS,5500 LEESVILLE RD,2060372,797622,...,98,1949.288940,I,no,D1,124,124,None,1.472133e+05,1949.288667
294180,21423205,21039914,11/19/2021,5:41:45PM,Wireless 911 Call,3,HIT AND RUN,FLETCHERS CHAPEL RD/BRIGHTWOOD LN,2058319,823738,...,48,8026.809082,I,no,D1,121,121,None,2.376268e+06,8026.808931


In [126]:
#Print merged data set info
dpd_w_policebeats.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 121823 entries, 0 to 317694
Data columns (total 26 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   Event Numb   121823 non-null  int64   
 1   Case Numbe   33727 non-null   object  
 2   Call Date    121823 non-null  object  
 3   Call Time    121823 non-null  object  
 4   Source       121823 non-null  object  
 5   Priority     121714 non-null  object  
 6   Nature       121823 non-null  object  
 7   Address      121823 non-null  object  
 8   X            121823 non-null  int64   
 9   Y            121823 non-null  int64   
 10  District     121734 non-null  object  
 11  Beat         121733 non-null  object  
 12  Dispositio   121823 non-null  object  
 13  Cancelled    121823 non-null  int64   
 14  geometry     121823 non-null  geometry
 15  index_right  121823 non-null  int64   
 16  OBJECTID     121823 non-null  int64   
 17  PERIMETER    121823 non-null  float64 
 

In [127]:
#Summary statistics per beat with visualizations

In [128]:
#Merge population data